In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>'''

In [ ]:
#--------------------------- Libraries ---------------------------//
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import scipy, random, copy, h5py, pandas, math, csv, sys, os, pickle, re, json, warnings
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn import preprocessing
from numpy.random import RandomState
from scipy import ndimage
from scipy import stats
from scipy import ndimage
from datetime import datetime
import os.path as osp

#--------------------------- Users, directories and some routines ---------------------------//
path = os.getcwd()

sub_routines = os.path.join(repo_dir,'python', 'sub_routines')
sys.path.append(sub_routines)
from neural_predictor import neural_predictor
from Import_MenuParameters import Import_MenuParameters
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)






In [ ]:
repo_dir = "/Users/germanriveros/Documents/GitHub/QueVasaEstudiar"

# #--------------------------- Functions ---------------------------//
def loadOutcomes():
    outcomes = {}
    with open(repo_dir + '/python/Options_Outcomes_05_15_2018.csv', encoding = 'utf-8' ) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            outcomes[int(row['Option_ID'])] = row
    return outcomes

def arange_outcomes(outcomes):
    HEOptions = {
    'MajorName'       : np.array([outcomes[x]['Option_Major_Name'] for x in outcomes.keys()]),
    'OptionID'        : np.array([x for x in outcomes.keys()]).astype(int),
    'InstCode'        : np.array([outcomes[x]['Option_Institution_Code'] for x in outcomes.keys()]).astype(int) -1,
    'Cutoff'          : np.array([outcomes[x]['LowerBoundScore'] for x in outcomes.keys()]).astype(float),
    'NoSearch'        : np.array([outcomes[x]['NoSearch'] for x in outcomes.keys()]).astype(int),
    'LevelCode'       : np.array([outcomes[x]['Option_Level_Code'] for x in outcomes.keys()]).astype(int) -1,
    'AreaCode'        : np.array([outcomes[x]['Option_Area_Code'] for x in outcomes.keys()]).astype(int) -1,
    'LocationCode'    : np.array([outcomes[x]['Option_Location_Code'] for x in outcomes.keys()]).astype(int) -1,
    'MajorCode'       : np.array([outcomes[x]['Option_Major_Code'] for x in outcomes.keys()]).astype(int) -1,
    'AverageEarning'  : np.array([outcomes[x]['Outcome_IncomeAve'] if outcomes[x]['Outcome_IncomeAve'] != "" else "-999" for x in outcomes.keys()]).astype(float)
    }

    HEOptions['AreaID']     = np.array(list(set(HEOptions['AreaCode'])))
    HEOptions['InstID']     = np.array(list(set(HEOptions['InstCode'])))      #We have to change this part
    HEOptions['LevelID']    = np.array(list(set(HEOptions['LevelCode'])))
    HEOptions['LocationID'] = np.array(list(set(HEOptions['LocationCode'])))
    HEOptions['MajorID']    = np.array(list(set(HEOptions['MajorCode'])))

    #Fixes:
    aux_InstID = np.array(range(HEOptions['InstID'].shape[0]))

    for y in HEOptions:
        HEOptions[y] = HEOptions[y][:,np.newaxis]

    for i in range(HEOptions['InstID'].shape[0]):
        HEOptions['InstCode'][HEOptions['InstCode'] == HEOptions['InstID'][i,0]] = aux_InstID[i]

    HEOptions['RelevantInst']  = aux_InstID[:,np.newaxis]
    HEOptions['normW']         = 10000000
    HEOptions['AverageEarning'][HEOptions['AverageEarning'] == -999] = np.nan

    return HEOptions

# #--------------------------- Data Arrange ---------------------------//

#1.1. IMPORT DATA ABOUT OPTIONS:
outcomes = loadOutcomes()
HEOptions = arange_outcomes(outcomes)

# 1.3. IMPORT DATA ABOUT ALL INTERACTIONS:
allinteractions = pd.read_csv(data_dir +'/chatbotprogress_data/allinteractions.csv', error_bad_lines=False)
allinteractions = allinteractions[(allinteractions.user != 'default-user') & (allinteractions.user.str.len() == 20)].reset_index(drop=True)
allinteractions["year"] = allinteractions.timestamp.str[0:4].astype(int)
allinteractions["month"] = allinteractions.timestamp.str[5:7].astype(int)
allinteractions["day"] = allinteractions.timestamp.str[8:10].astype(int)
allinteractions["second"] = allinteractions.timestamp.str[17:19].astype(int)
allinteractions["minute"] = allinteractions.timestamp.str[14:16].astype(int)
allinteractions["hour"] = allinteractions.timestamp.str[11:13].astype(int) - 7
allinteractions["hour"][allinteractions["hour"]==-7] = 17
allinteractions["hour"][allinteractions["hour"]==-6] = 18
allinteractions["hour"][allinteractions["hour"]==-5] = 19
allinteractions["hour"][allinteractions["hour"]==-4] = 20
allinteractions["hour"][allinteractions["hour"]==-3] = 21
allinteractions["hour"][allinteractions["hour"]==-2] = 22
allinteractions["hour"][allinteractions["hour"]==-1] = 23

allinteractions = allinteractions[(allinteractions.year == 2018) & (allinteractions.month >= 9)]
allinteractions = allinteractions[(allinteractions.user != "992") & (allinteractions.user != "YBOF9JBM8DE9ME6RE1K4") & (allinteractions.user != "EMF393CY0PX7ESW0BNQ")]
allinteractions = allinteractions.sort_values(['month','day'])
allinteractions['Cummulative'] = np.cumsum(np.ones((allinteractions.shape[0],1))).astype(int)
allinteractions['Interacted'] = 1
allinteractions['times'] = list(map(lambda x: x[0:10] + " " + x[11:19], allinteractions['timestamp']))
allinteractions['times'] = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), allinteractions['times']))
allinteractions['hours'] = list(map(lambda x: x[0:10] + " " + x[11:13], allinteractions['timestamp']))
allinteractions['hours'] = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d %H'), allinteractions['hours']))
allinteractions['days'] = list(map(lambda x: x[0:10] , allinteractions['timestamp']))
allinteractions['days'] = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d'), allinteractions['days']))

#1.3. IMPORT DATA ABOUT WAGE DEVIATIONS:
wageDeviation   = pd.read_csv(data_dir +'/chatbotprogress_data/wage_deviation.csv'.format(100)).drop(columns = ['Unnamed: 0']).rename(columns= {'studentID': 'user'})
wageDeviation['hours'] = list(map(lambda x: x[0:10] + " " + x[11:13], wageDeviation['time']))
wageDeviation['hours'] = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d %H'), wageDeviation['hours']))
wageDeviation['times'] = list(map(lambda x: x[0:10] + " " + x[11:19], wageDeviation['time']))
wageDeviation['times'] = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), wageDeviation['times']))
wageDeviation['days'] = list(map(lambda x: x[0:10] , wageDeviation['time']))
wageDeviation['days'] = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d'), wageDeviation['days']))
wageDeviation['interaction'] = 1
wageDeviation['absWageDeviation'] = list(map(lambda x: abs(x), wageDeviation['wagedeviation']))


# Number of Menus by user
wageDeviation = wageDeviation.sort_values(by = ['user', 'times'])
wageDeviation['numberMenu'] = -999
wageDeviation['numberMenu'] = wageDeviation.groupby('user')['times'].rank(ascending=True).astype(int)

# No vectorized code
# for i in range(1,wageDeviation.shape[0]):
#     if wageDeviation['user'].iloc[i] == wageDeviation['user'].iloc[i-1]:
#         j += 1
#         wageDeviation['numberMenu'].iloc[i] = j
#     else:
#         j = 1
#         wageDeviation['numberMenu'].iloc[i] = j

wageDeviation['numberMenu'][wageDeviation['numberMenu'] == 8] = 8
wageDeviation['numberMenu'][wageDeviation['numberMenu'] == 9] = 8
wageDeviation['numberMenu'][wageDeviation['numberMenu']>=10] = 9
wageDeviation['N_total']=1


#=================Data for performance of the Structural and ML model====================
parameters = {}
path_pickle = repo_dir + "/python/data/NN_Parameters"
with open(path_pickle + '/parameters_nn.pickle', 'rb') as handle:
    parameters['NN'] = pickle.load(handle)

studentFeatures = pd.read_csv(repo_dir + '/Student_Features_Fall2018.csv'.format(100))
interactionsBot = pd.read_csv(data_dir + '/chatbotprogress_data/interactions_bot.csv'.format(100))
seedAssignment  = pd.read_csv(data_dir +'/chatbotprogress_data/seeds_intervention.csv').drop(columns=['Unnamed: 0'])
interactions    = seedAssignment.merge(studentFeatures, left_on = 'url_id', right_on = 'url_id', how = 'left')
interactions    = interactions.merge(interactionsBot, left_on = 'Student_ID', right_on = 'Student_ID', how = 'inner')

n_int = interactions.shape[0]
n_options = HEOptions['MajorName'].shape[0]

#2.2. Generate tags
dpto_code = list(set(studentFeatures['Student_Location_Code']))

for x in dpto_code:
    interactions['dpto_'+str(x)] = (interactions['Student_Location_Code'] == x).astype(int)
    dpto_tag = ['dpto_'+str(x) for x in dpto_code]

menu_tag = ['Menu1','Menu2','Menu3','Menu4']
tag_continuous = ['punt_rez_sede','inse_rez_sede','puntsd_rez_sede', 'insesd_rez_sede', 'edad']

X_indexslates = np.array(interactions[menu_tag])
for i,j in zip(range(n_options), list(HEOptions['OptionID'][:,0])):
    X_indexslates[:,0][X_indexslates[:,0]==j] = i
    X_indexslates[:,1][X_indexslates[:,1]==j] = i
    X_indexslates[:,2][X_indexslates[:,2]==j] = i
    X_indexslates[:,3][X_indexslates[:,3]==j] = i

X_SlateEarnigs          = HEOptions['AverageEarning'][X_indexslates].reshape(n_int,len(menu_tag))
X_SlateEarnigs[np.isnan(X_SlateEarnigs)] = np.nanmean(X_SlateEarnigs)
X_SlateEarnigs          = X_SlateEarnigs
X_SlateLevels           = HEOptions['LevelCode'][X_indexslates].reshape(n_int,len(menu_tag))
X_SlateInst             = HEOptions['InstCode'][X_indexslates].reshape(n_int,len(menu_tag))
X_SlateArea             = HEOptions['AreaCode'][X_indexslates].reshape(n_int,len(menu_tag))
X_Score                 = np.array(interactions['Student_Score']).reshape(n_int,1)
X_ScoreDecil            = np.array(interactions['Student_ScoreDecil']).reshape(n_int,1)
X_Continuous            = np.array(interactions[tag_continuous]).reshape(n_int,len(tag_continuous)).astype(int)
X_Gender                = np.array(interactions['Student_Gender']).reshape(n_int,1)
X_LocationCode          = np.array(interactions[dpto_tag]).reshape(n_int,len(dpto_tag)).astype(int)

X                       = np.hstack((X_SlateEarnigs,X_SlateLevels,X_SlateInst,X_SlateArea,X_Gender,X_Score,X_ScoreDecil,X_LocationCode,X_Continuous))
Y                       = np.array(interactions['Selection']).reshape(n_int, 1) == np.concatenate((np.array(interactions[['Menu1','Menu2','Menu3','Menu4']]), np.array([-999]*n_int).reshape(n_int,1)), axis =  1)

X                       = (X - parameters['NN']['X_mean'])/parameters['NN']['X_std']


ClickProb, _ = neural_predictor(X.T, parameters['NN'], 'NN', None, None)
Y_hat = ClickProb.T == ClickProb.T.max(1).reshape(n_int,1)

#2. How does the neural network perform on the data provided by the different treatment arms??

#2.1. Accuracy
Accuracy = (Y * Y_hat).sum(1).reshape(n_int,1)

#2.2. Probability
Pr = (ClickProb.T * (Y * Y_hat)).sum(1).reshape(n_int,1)

#2.3. Expected Value:
Pr_dist = ClickProb.T[:,:-1]/ClickProb.T[:,:-1].sum(1).reshape(n_int,1)
EV = (X_SlateEarnigs*Pr_dist).sum(1).reshape(n_int,1)/10**7

#3. Compute cool graphics:
#3.1 Create DataFrame that allows us to create cool graphics:
TreatmentArm = np.array(interactions['bot']).reshape(n_int,1)
NeuralNet_DataFrame = pd.DataFrame(np.hstack((TreatmentArm, Accuracy, Pr, EV)), columns = ['TreatmentArm', 'Accuracy','Probability','ExpectedValue']).astype(float)
NeuralNet_DataFrame.TreatmentArm = NeuralNet_DataFrame.TreatmentArm.astype(int)

bot_map = {
    '11' : ('Econ',   'Unrestricted'),
    '12' : ('Econ',   'Diverse'),
    '13' : ('Econ',   'Targeted'),
    '21' : ('ML',     'Unrestricted'),
    '22' : ('ML',     'Diverse'),
    '23' : ('ML',     'Targeted'),
    '31' : ('Random', 'Unrestricted'),
    '32' : ('Random', 'Diverse'),
    '33' : ('Random', 'Targeted')
 }

bot_types = ['11','12','13','21','22','23','31','32','33']
bot_label = []
for x in bot_types: bot_label.append(bot_map[x][0][:2] +': ' + bot_map[x][1][0])

NeuralNet_DataFrame['TreatmentArm_Label'] = 0
for x in bot_types:
    _label  = bot_map[x][0][:2] +': ' + bot_map[x][1][0]
    NeuralNet_DataFrame['TreatmentArm_Label'][NeuralNet_DataFrame['TreatmentArm'] == int(x)] = _label

In [ ]:
# Chatbot Daily Progress
gcolor = '#e6550d'

# Cummulative progress of the chatbot
# ax = sns.lineplot(x="times", y="Cummulative", data=allinteractions, color=gcolor)
# ax.set_xlabel('')
# for tick in ax.get_xticklabels(): tick.set_rotation(45)
# plt.show()

In [ ]:
# Frequency of interactions
ax = sns.lineplot(x="hours", y="Interacted", data=allinteractions.groupby('hours').sum().reset_index(), color=gcolor)
ax.set_xlabel('')
for tick in ax.get_xticklabels(): tick.set_rotation(45)
plt.show()

In [ ]:
# Type of interaction by hour
last_date = list(set(wageDeviation['days']))
last_date.sort()
last_date = str(last_date[-1])


# Wage deviation from true value by day
fig, ax1 = plt.subplots()
ax = sns.lineplot(x="days", y="interaction", data=wageDeviation[wageDeviation['days']<last_date].groupby(['days']).sum().reset_index(), color='blue')
ax.set_xlabel('')
ax.set_ylabel('Number of interactions', color='tab:blue')
ax.tick_params(axis='y', labelcolor='tab:blue')
ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
ax2 = sns.lineplot(x="days", y="absWageDeviation", data=wageDeviation[wageDeviation['days']<last_date], color='red')
ax2.set_ylabel('Wage Differential', color='tab:red')  # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor='tab:red')
for tick in ax.get_xticklabels(): tick.set_rotation(45)
plt.show()

In [ ]:
# Wage deviation from true value by number of menu interaction¶
fig, ax = plt.subplots()
ax = sns.lineplot(x="numberMenu", y="absWageDeviation", data=wageDeviation[wageDeviation['days']<last_date], color=gcolor)
ax.set_xlabel('Number of Menu')
ax.set_ylabel('Percentage deviation from true value', color='tab:blue')
ax.tick_params(axis='y', labelcolor='tab:blue')
ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
ax2 = sns.lineplot(x="numberMenu", y="N_total", data=wageDeviation[wageDeviation['days']<last_date].groupby(['numberMenu']).sum().reset_index(), color='gray')
ax2.set_ylabel('Number of observations', color='tab:red')  # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor='tab:red')
labels = [int(x) for x in ax.get_xticks().tolist()]
labels[-2]='>9'
ax.set_xticklabels(labels)
plt.show()

In [ ]:
# Number of students assigned to each treatment¶
seedAssignment['count'] = 1
ax = sns.barplot(x="bot", y="count", data=seedAssignment.groupby(['bot']).sum().reset_index(), capsize=.2)
ax.set_xlabel('')
ax.set_ylabel('Number of treatments assigned')
ax.set_xticklabels(bot_label)
plt.show()

In [ ]:
# Number of outside option clicks¶
interactions['outsideOption'] = (interactions['Selection'] == -999)
ax = sns.barplot(x="bot", y="outsideOption", data=interactions.groupby(['bot']).sum().reset_index(), capsize=.2)
ax.set_xlabel('')
ax.set_ylabel('Number of outside option clicks')
ax.set_xticklabels(bot_label)
plt.show()

In [ ]:
# Percentage of outside option selected¶
interactions['outsideOption'] = (interactions['Selection'] == -999)
ax = sns.barplot(x="bot", y="outsideOption", data=interactions.groupby(['bot']).mean().reset_index(), capsize=.2)
ax.set_xlabel('')
ax.set_ylabel('Number of outside option clicks')
ax.set_xticklabels(bot_label)
plt.show()

In [ ]:
# Number of menus shown by treatment arm:
fig, ax = plt.subplots()
interactions['count'] = 1
ax = sns.barplot(x="bot", y="count", data=interactions.groupby(['bot']).sum().reset_index(), capsize=.2)
ax.set_xlabel('')
ax.set_ylabel('Number of menus showed by treatment')
ax.set_xticklabels(bot_label)
plt.show()